# Extract Lemmatization from JSON: Basic Parser
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract lemmatization data for one or more texts. The resulting `csv` (Comma Separated Values) file has two fields: a Text ID (e.g. `dcclt/Q000039`) and a string of lemmas in the format `lugal[king]N` (or `šarru[king]N` for Akkadian texts).

The `JSON` files are available in a zipped format on [ORACC](http://oracc.org). They must be downloaded from [ORACC](http://oracc.org) and put in a directory called `jsonzip`. You may use the script [download_ORACC-JSON.ipynb](download_ORACC-JSON.ipynb) or download the files by hand.

The output of the Basic Parser does not recognize lines (all words/lemmas in a text are listed sequentially) and does not indicate breakage. These issues are of little consequence for [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) techniques such as [topic modeling](https://en.wikipedia.org/wiki/Topic_model), but may become serious limitations for other types of computational approaches. The Advanced Parser will introduce techniques to extract such data.

[ORACC](http://oracc.org) `JSON` files also include data on the sign level (reading, sign name, unicode number, function, etc). These data are not extracted in the parsers demonstrated here, but the example code should provide a model for how any type of data can be extracted.

In [1]:
import pandas as pd
import zipfile
import json

## 1.1 Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `text_ids`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

In [2]:
filename = input('Filename: ')

Filename: test.txt


## 1.2 Open the List of Text IDs and Remove Spaces and Empty Lines
The list of text IDs is read; accidental space at the beginning or end of each line are removed, as well as blank lines. The Advanced Parser can read a partial text by indicating the beginning and the end of the desired text segment and this is also indicated in the list of text IDs in the format `PROJECT/TEXTID start line - stop line`. Since the Basic Parser only parses full texts, we strip everything after the first space.


In [3]:
textids = 'text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]        # strip spaces left and right
pqxnos = [x for x in pqxnos if not x == ""] # strip empty lines
pqxnos = [x.split()[0] for x in pqxnos]     # strip everything after the first space

## 2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For nore information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.museum.upenn.edu/doc/opendata/index.html).

The first argument of the `parsejson()` function is a `JSON` object, a dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` which itself contains an array (a list) of `JSON` objects. Iterating through these objects, if an object contains another `cdl` node, the function calls itself with this object as first argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the value of that key is appended to the list `lemm_l`. This list is defined outside of the function proper. 

The field `id_text` is derived from the input list. It consists of a project abbreviation, such as `blms` or `cams/gkab` plus a text ID, in the format `cams/gkab/P338616` or `dcclt/Q000039`. The `id_text` is added as a second argument to the function and is added to the lemmatization data of every word.

In [4]:
def parsejson(text, id_text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject, id_text)
        if "f" in JSONobject:
            lemm = JSONobject["f"]
            lemm["id_text"] = id_text
            lemm_l.append(lemm)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will read the list of text IDs identified above. Each text ID (e.g.`dcclt/Q000039`) is split into the ID number (`Q000039`) and the project designation (`dcclt`). The project designation is used to identify the right ZIP file in the directory `jsonzip`. The ID number is used to find the pertinent `json` file inside the ZIP.

The command `json.loads()` reads the json data and transforms it into a Python dictionary (a sequence of keys and values).

This dictionary, which is called `text` is now sent to the `parsejson()` function, with the text ID as second argument. The function adds lemmata to the `lemm_l` list.

In [5]:
lemm_l = []
for id_text in pqxnos:
    project = id_text[:-8].lower()
    pqx = id_text[-7:].upper()
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    data = z.read(project + "/corpusjson/" + pqx +".json")
    data_json = json.loads(data)
    try:
        print("parsing " + id_text)
        parsejson(data_json, id_text)
    except:
        print(id_text + ' is not available or not complete')

parsing dcclt/Q000057
parsing blms/P282597
parsing blms/P274259
parsing saao/saa04/P237549
jsonzip/saao-saa21.zip does not exist or is not a proper ZIP file


In [6]:
word_df = pd.DataFrame(lemm_l).fillna('')
word_df

base       cf cont delim epos               form  \
0     dirig    dirig             V/i              dirig   
1                                                     x   
2                                                |SI.A|   
3              watru              AJ        wa-at-ru-um   
4     dirig    dirig             V/i              dirig   
5             šūturu              AJ        šu-tu-ru-um   
6     dirig    dirig             V/i              dirig   
7            neqelpû               N    ni-qe₃-el-pu-um   
8     dirig    dirig             V/i              dirig   
9              ašāšu               N           a-ša-šum   
10    dirig    dirig             V/i              dirig   
11               eli             PRP               e-li   
12                                                dirig   
13                                                 el-x   
14    dirig    dirig             V/i              dirig   
15            kāpidu               N        ka-pi-du-um   
16    dirig    dirig             V/i              dirig   
17            zāqiru               N      sa₆-qi₃-ru-um   
18    dirig    dirig             V/i              dirig   
19              waṣû               N       wa-ṣu₂-u₂-um   
20    dirig    dirig             V/i              dirig   
21            izuzzu               N      i-zu-uz-zu-um   
22    dirig    dirig             V/i              dirig   
23             adāru               N     na-ah-du-ru-um   
24    dirig    dirig             V/i              dirig   
25            qarāru               N  na-ga-ar-ru-ru-um   
26    dirig    dirig               N              dirig   
27               amu               N            a-mu-um   
28    dirig    dirig               N              dirig   
29              hāmu               N           ha-mu-um   
...     ...      ...  ...   ...  ...                ...   
2279                                               i-na   
2280                                                ŠA₃   
2281                                          UDU.NITA₂   
2282                                               NE-i   
2283                                       GUB-za-am-ma   
2284                                              an-na   
2285                                              GI.NA   
2286                                        GIŠ.HUR-MEŠ   
2287                                          SILIM-MEŠ   
2288                                            UZU-MEŠ   
2289                                             ta-mit   
2290                                           SIG₅-MEŠ   
2291                                          SILIM-MEŠ   
2292                                                ša₂   
2293                                          SILIM-tim   
2294                                                ša₂   
2295                                                 KA   
2296                                       DINGIR-ti-ka   
2297                                             GAL-ti   
2298                                         šuk-nam-ma   
2299                                             lu-mur   
2300                                                UGU   
2301                                       DINGIR-ti-ka   
2302                                             GAL-ti   
2303                                             {d}UTU   
2304                                                 EN   
2305                                             GAL-u₂   
2306                                         lil-lik-ma   
2307                                                KIN   
2308                                         li-tap-pal   

                                                    gdl                   gw  \
0               [{'v': 'dirig', 'id': 'Q000057.1.1.0'}]               exceed   
1     [{'x': 'ellipsis', 'id': 'Q000057.1.2.0', 'bre...                        
2     [{'c': '|SI.A|', 'id': 'Q000057.1.3.0', 'seq':...                        
3     [{'v': 'wa', 'id': 'Q0

## 3.1 Create a `lemma` column
The following code combines the `cf` (Citation Form), `gw` (Guide Word), and `pos` (Part of Speech) columns to create a new `lemma` column with the format `cf[gw]pos`, for instance `šarru[king]N` or `lugal[king]N`. Unlemmatized words do not have `cf`, `gw`, or `pos` - they only have `form` (the transliteration). The function therefore has a condition: if `cf` is empty, the format should be `form[NA]NA`. Alternatively, one may leave out non-lemmatized words altogether and create the `lemma` column by simply adding up `cf`, `gw`, and `pos`, as follows:

> `word_df = word_df[word_df['cf'] != '']`

> `word_df['lemma'] = word_df['cf'] + '[' + word_df['gw'] + ']' + word_df['pos']`

In [7]:
word_df["lemma"] = word_df.apply(lambda r: (r["cf"] + '[' + r["gw"] + ']' + r["pos"]) if r["cf"] != '' 
                                 else r['form'] + '[NA]NA', axis=1)
word_df[['id_text', 'lemma']]

id_text                        lemma
0          dcclt/Q000057             dirig[exceed]V/i
1          dcclt/Q000057                      x[NA]NA
2          dcclt/Q000057                 |SI.A|[NA]NA
3          dcclt/Q000057                watru[huge]AJ
4          dcclt/Q000057             dirig[exceed]V/i
5          dcclt/Q000057         šūturu[very great]AJ
6          dcclt/Q000057              dirig[float]V/i
7          dcclt/Q000057              neqelpû[float]V
8          dcclt/Q000057               dirig[fall]V/i
9          dcclt/Q000057  ašāšu[be(come) distressed]V
10         dcclt/Q000057             dirig[exceed]V/i
11         dcclt/Q000057                   eli[on]PRP
12         dcclt/Q000057                  dirig[NA]NA
13         dcclt/Q000057                   el-x[NA]NA
14         dcclt/Q000057             dirig[exceed]V/i
15         dcclt/Q000057             kāpidu[planner]N
16         dcclt/Q000057             dirig[exceed]V/i
17         dcclt/Q000057    zāqiru[one who projects]N
18         dcclt/Q000057             dirig[exceed]V/i
19         dcclt/Q000057                waṣû[go out]V
20         dcclt/Q000057             dirig[exceed]V/i
21         dcclt/Q000057               izuzzu[stand]V
22         dcclt/Q000057               dirig[fall]V/i
23         dcclt/Q000057        adāru[be(come) dark]V
24         dcclt/Q000057             dirig[exceed]V/i
25         dcclt/Q000057              qarāru[writhe]V
26         dcclt/Q000057                 dirig[raft]N
27         dcclt/Q000057                   amu[raft]N
28         dcclt/Q000057                 dirig[raft]N
29         dcclt/Q000057                  hāmu[raft]N
...                  ...                          ...
2279  saao/saa04/P237549                   i-na[NA]NA
2280  saao/saa04/P237549                    ŠA₃[NA]NA
2281  saao/saa04/P237549              UDU.NITA₂[NA]NA
2282  saao/saa04/P237549                   NE-i[NA]NA
2283  saao/saa04/P237549           GUB-za-am-ma[NA]NA
2284  saao/saa04/P237549                  an-na[NA]NA
2285  saao/saa04/P237549                  GI.NA[NA]NA
2286  saao/saa04/P237549            GIŠ.HUR-MEŠ[NA]NA
2287  saao/saa04/P237549              SILIM-MEŠ[NA]NA
2288  saao/saa04/P237549                UZU-MEŠ[NA]NA
2289  saao/saa04/P237549                 ta-mit[NA]NA
2290  saao/saa04/P237549               SIG₅-MEŠ[NA]NA
2291  saao/saa04/P237549              SILIM-MEŠ[NA]NA
2292  saao/saa04/P237549                    ša₂[NA]NA
2293  saao/saa04/P237549              SILIM-tim[NA]NA
2294  saao/saa04/P237549                    ša₂[NA]NA
2295  saao/saa04/P237549                     KA[NA]NA
2296  saao/saa04/P237549           DINGIR-ti-ka[NA]NA
2297  saao/saa04/P237549                 GAL-ti[NA]NA
2298  saao/saa04/P237549             šuk-nam-ma[NA]NA
2299  saao/saa04/P237549                 lu-mur[NA]NA
2300  saao/saa04/P237549                    UGU[NA]NA
2301  saao/saa04/P237549           DINGIR-ti-ka[NA]NA
2302  saao/saa04/P237549                 GAL-ti[NA]NA
2303  saao/saa04/P237549                 {d}UTU[NA]NA
2304  saao/saa04/P237549                     EN[NA]NA
2305  saao/saa04/P237549                 GAL-u₂[NA]NA
2306  saao/saa04/P237549             lil-lik-ma[NA]NA
2307  saao/saa04/P237549                    KIN[NA]NA
2308  saao/saa04/P237549             li-tap-pal[NA]NA

[2309 rows x 2 columns]

## 3.2 Remove Spaces and Commas from the Lemma
Spaces and commas in the Guide Word may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [8]:
word_df.loc[:,'lemma'] = [x.replace(' ', '-') for x in word_df.loc[:,'lemma']]
word_df.loc[:,'lemma'] = [x.replace(',', '') for x in word_df.loc[:,'lemma']]
word_df

base       cf cont delim epos               form  \
0     dirig    dirig             V/i              dirig   
1                                                     x   
2                                                |SI.A|   
3              watru              AJ        wa-at-ru-um   
4     dirig    dirig             V/i              dirig   
5             šūturu              AJ        šu-tu-ru-um   
6     dirig    dirig             V/i              dirig   
7            neqelpû               N    ni-qe₃-el-pu-um   
8     dirig    dirig             V/i              dirig   
9              ašāšu               N           a-ša-šum   
10    dirig    dirig             V/i              dirig   
11               eli             PRP               e-li   
12                                                dirig   
13                                                 el-x   
14    dirig    dirig             V/i              dirig   
15            kāpidu               N        ka-pi-du-um   
16    dirig    dirig             V/i              dirig   
17            zāqiru               N      sa₆-qi₃-ru-um   
18    dirig    dirig             V/i              dirig   
19              waṣû               N       wa-ṣu₂-u₂-um   
20    dirig    dirig             V/i              dirig   
21            izuzzu               N      i-zu-uz-zu-um   
22    dirig    dirig             V/i              dirig   
23             adāru               N     na-ah-du-ru-um   
24    dirig    dirig             V/i              dirig   
25            qarāru               N  na-ga-ar-ru-ru-um   
26    dirig    dirig               N              dirig   
27               amu               N            a-mu-um   
28    dirig    dirig               N              dirig   
29              hāmu               N           ha-mu-um   
...     ...      ...  ...   ...  ...                ...   
2279                                               i-na   
2280                                                ŠA₃   
2281                                          UDU.NITA₂   
2282                                               NE-i   
2283                                       GUB-za-am-ma   
2284                                              an-na   
2285                                              GI.NA   
2286                                        GIŠ.HUR-MEŠ   
2287                                          SILIM-MEŠ   
2288                                            UZU-MEŠ   
2289                                             ta-mit   
2290                                           SIG₅-MEŠ   
2291                                          SILIM-MEŠ   
2292                                                ša₂   
2293                                          SILIM-tim   
2294                                                ša₂   
2295                                                 KA   
2296                                       DINGIR-ti-ka   
2297                                             GAL-ti   
2298                                         šuk-nam-ma   
2299                                             lu-mur   
2300                                                UGU   
2301                                       DINGIR-ti-ka   
2302                                             GAL-ti   
2303                                             {d}UTU   
2304                                                 EN   
2305                                             GAL-u₂   
2306                                         lil-lik-ma   
2307                                                KIN   
2308                                         li-tap-pal   

                                                    gdl                   gw  \
0               [{'v': 'dirig', 'id': 'Q000057.1.1.0'}]               exceed   
1     [{'x': 'ellipsis', 'id': 'Q000057.1.2.0', 'bre...                        
2     [{'c': '|SI.A|', 'id': 'Q000057.1.3.0', 'seq':...                        
3     [{'v': 'wa', 'id': 'Q0

## 3.3 Group by Textid
Get all the lemmas that belong to a single text in one row (one row = one document). The `agg()` (aggregate) function, which works on the result of a `groupby()` process aggregates columns of the original dataframe. The function takes a dictionary in which the keys are column names and the values are functions to be used in the aggregation process. The example below has only one such function (`' '.join` will join all entries in the colum `lemma` with a space in between); one may specify (the same or different) functions for different columns, for instance:
> word_df = word_df.groupby("textid").agg({"lemma": ' '.join, "base": ' '.join})

In [9]:
word_df = word_df.groupby("id_text").agg({"lemma": ' '.join})
word_df

lemma
id_text                                                              
blms/P274259        x-x-šu₂[NA]NA x[NA]NA isqu[share]N x[NA]NA x[N...
blms/P282597        {d}utu-gin₇[NA]NA e₃-ta[NA]NA uru₂-zu[NA]NA e-...
dcclt/Q000057       dirig[exceed]V/i x[NA]NA |SI.A|[NA]NA watru[hu...
saao/saa04/P237549  {d}UTU[NA]NA EN[NA]NA GAL-u₂[NA]NA ša₂[NA]NA a...

## 4 Save Results in CSV file
The `.csv` file has the same name as the list of textid's that was used at the beginning of this notebook. In most computers, `csv` files open automatically in Excel. This spread sheet program does not deal well with `utf-8` encoding. If you intend to use the file in Excel, change `encoding ='utf-8'` to `encoding='utf-16'`. For usage in computational text analysis applications `utf-8` is usually preferred.

In [10]:
savefile =  filename[:-3] + 'csv'
with open('output/' + savefile, 'w') as w:
    word_df.to_csv(w, encoding = 'utf-8')